In [6]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import init
import functools
from torch.optim import lr_scheduler
import torch.nn.functional as F
import time
from options.test_options import TestOptions, Opt_test
from data import DataLoader
from models import create_model
from util.writer import Writer
from test import run_test
from util.mesh_viewer import *
from util.util import print_network


In [7]:
opt = Opt_test()
opt.parse()
# These options must match the options for training!
opt.dataroot = 'datasets/shrec_16' 
opt.name = 'shrec16'
opt.ncf = [64, 128, 256, 256] 
opt.pool_res = [600, 450, 300, 180]
opt.norm = 'group'
opt.resblocks = 1 
opt.export_folder = 'stuff'


dataset = DataLoader(opt)
dataset_size = len(dataset)
print('#test meshes = %d' % dataset_size)
model = create_model(opt)
writer = Writer(opt)

def test_all(opt, dataset, model, writer, epoch=-1):
    print('Running Test')
    #opt = TestOptions().parse()
    opt.serial_batches = True  # no shuffle
    #dataset = DataLoader(opt)
    #model = create_model(opt)
    
    # test
    writer.reset_counter()
    for i, data in enumerate(dataset):
        model.set_input(data)
        ncorrect, nexamples = model.test()
        writer.update_counter(ncorrect, nexamples)
    writer.print_acc(epoch, writer.acc)
    return writer.acc

def test_random(opt, dataset, model):
    num_obj = np.random.randint(len(dataset))
    opt.serial_batches = True 
    for i, data in enumerate(dataset):
            if i == num_obj//16:
                idx = num_obj - num_obj//16*16
                model.set_input(data)
                out = model.forward()
                pred_class = out.data.max(1)[1]
                #ncorrect, nexamples = model.test()
                print('Correct class: %s' % dataset.dataset.classes[data['label'][idx]])
                print('Predicted class: %s' % dataset.dataset.classes[pred_class[idx]])
        
                obj_name = dataset.dataset.paths[num_obj][0]
                print('see object file %s for visualization' % obj_name.split('\\')[-1])
            

loaded mean / std from cache
#test meshes = 120
loading the model from ./checkpoints\shrec16\latest_net.pth


In [3]:
 test_all(opt, dataset, model, writer, epoch=-1)

Running Test
epoch: -1, TEST ACC: [98.333 %]



0.9833333333333333

In [4]:
test_random(opt, dataset, model)

Correct class: armadillo
Predicted class: armadillo
see object file T473.obj for visualization


In [ ]:
dataset.dataset.classes
ting = dataset.dataset.paths[0][0]
view_meshes(ting, dataset.dataset.paths[0][0])

In [13]:
print_network(model.net)

---------- Network initialized -------------
[Network] Total number of parameters : 1.323 M
-----------------------------------------------


In [12]:
model.net

MeshConvNet(
  (conv0): MResConv(
    (conv0): MeshConv(
      (conv): Conv2d(5, 64, kernel_size=(1, 5), stride=(1, 1), bias=False)
    )
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1): MeshConv(
      (conv): Conv2d(64, 64, kernel_size=(1, 5), stride=(1, 1), bias=False)
    )
  )
  (norm0): GroupNorm(16, 64, eps=1e-05, affine=True)
  (pool0): MeshPool()
  (conv1): MResConv(
    (conv0): MeshConv(
      (conv): Conv2d(64, 128, kernel_size=(1, 5), stride=(1, 1), bias=False)
    )
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1): MeshConv(
      (conv): Conv2d(128, 128, kernel_size=(1, 5), stride=(1, 1), bias=False)
    )
  )
  (norm1): GroupNorm(16, 128, eps=1e-05, affine=True)
  (pool1): MeshPool()
  (conv2): MResConv(
    (conv0): MeshConv(
      (conv): Conv2d(128, 256, kernel_size=(1, 5), stride=(1, 1), bias=False)
    )
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, 